In [1]:
# _list = []
# for eachFile in os.listdir(_DATA_PATH):
#     if ('Msg' not in eachFile) and (eachFile.endswith('json')):
#         _list.append(pd.read_json(os.path.join(_DATA_PATH,eachFile)))
import os
import json
import glob
from pathlib import Path

# please change the directory
_DATA_PATH = r'C:\Users\MAXMAK\Downloads\WRL 20201214_1500-2159\WRL 20201214_1500-2159' 
_list = []

for f in Path(_DATA_PATH).rglob("*.json"):
    with open(f, 'rb') as file_in:
        _list.append(json.load(file_in))

with open("merged_file.json", "w") as outfile:
    json.dump(_list, outfile)

# Read file

In [2]:
with open(r'merged_file.json', 'r', encoding='UTF-8') as file_in:
    data_str = file_in.read()

In [3]:
len(data_str)

45175752

In [4]:
# print("string_head: ", data_str[:100])
# print("string_tail: ", data_str[-100:])

# Replace element

<!-- data_str = data_str.replace('true','True')
data_str = data_str.replace('false','False')
data_str = data_str.replace('null','None')
data_str = data_str.replace('\n','') -->

In [5]:
def query_type(data):
    for ch in data:
        if ch == '{':
            return "dict"
        if ch == '[':
            return "list"
    return "value"

# Check function type

In [6]:
data_head = []  # To ensure the order, so that dict can be used in mapping
def get_json_head(data, loc=""):
    data = str(data) # dtype as string

    data_type = query_type(data)
    if data_type == "value":  # element
        if loc[1:] not in data_head:
             data_head.append(loc[1:])
        return

    if data_type == "dict":  # dict
        data_dict = eval(data)
        for key in data_dict:
            get_json_head(data_dict[key], loc + "-" + key) # not necessary but ust "-" to distinguish the data level, can replace with other like "_"
        return

    if data_type == "list":  # list
        data_list = list(eval(data))
        for item in data_list:
            get_json_head(item, loc)
        return

get_json_head(data_str)
#print(data_head[:10])
#print(data_head[-10:])
print("# of data_head:", len(data_head))
print(data_head)

# of data_head: 12
['rtss-curr_time', 'rtss-line-id', 'rtss-line-stations-id', 'rtss-line-stations-track-id', 'rtss-line-stations-track-train-seq', 'rtss-line-stations-track-train-dest', 'rtss-line-stations-track-train-plat', 'rtss-line-stations-track-train-time', 'rtss-line-stations-track-train-ttnt', 'rtss-line-stations-track-train-valid', 'rtss-line-stations-track-train-td', 'rtss-line-stations-track-train-source']


In [7]:
_len = []

for each in data_head:
    _ = '\"'+ each.split("-")[-1] +'\"'
    print("{} : {}".format(_, data_str.count(_)))
    _len.append(data_str.count(_))
    
deepest_data_length = max(_len)

"curr_time" : 2836
"id" : 99260
"id" : 99260
"id" : 99260
"seq" : 322438
"dest" : 322438
"plat" : 322438
"time" : 322438
"ttnt" : 322438
"valid" : 322438
"td" : 322438
"source" : 322438


# Prepare the array

In [8]:
data_head_dict = {}
for head in data_head:
    # print(head)
    tmp = []
    for i in range(deepest_data_length):
#     for i in range(322438): # To expand the array length/ size brutally, i.e. guess how many data
        tmp.append("")
    data_head_dict[head] = tmp

In [9]:
# for key in data_head[:]:
#     # print(key, data_head_dict[key][:])
#     # print(key, len(data_head_dict[key]))
#     print(key, data_head_dict[key][:], len(data_head_dict[key]))
#     pass

In [10]:
row_now = 0
def get_json_table(data, loc="", rows=0):
    global row_now
    data = str(data)  # dtype as string

    data_type = query_type(data)
    if data_type == "value":  # element
        key = loc[1:]
        data_head_dict[key][rows] = data 
        return

    if data_type == "dict":  # dict
        data_dict = eval(data)
        for key in data_dict:
            get_json_table(data_dict[key], loc + "-" + key, rows)
        return

    if data_type == "list":  # list
        data_list = list(eval(data))
        for i in range(len(data_list)):
            if i > 0:
                row_now += 1
            get_json_table(data_list[i], loc, row_now)
        return

In [11]:
get_json_table(data_str)

for key in data_head[:]:
    # print(key, data_head_dict[key][:10])
    # print(key, len(data_head_dict[key]))
    print(key, data_head_dict[key][:], len(data_head_dict[key]))
    pass

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# output csv

In [12]:
with open('tmp.csv', 'w',encoding="gbk") as file_out:
    for head in data_head[:-1]:
        file_out.write(head)
        file_out.write(",")  # comma separate 
    file_out.write(data_head[-1] + "\n")  # last row next line
    for i in range(deepest_data_length):
#     for i in range(322438):
        for head in data_head[:-1]:
            file_out.write(data_head_dict[head][i])
            file_out.write(",")
        last_key = data_head[-1]  # get last head
        file_out.write(data_head_dict[last_key][i])
        file_out.write("\n")

In [13]:
import pandas as pd
import os

df = pd.read_csv('tmp.csv')
df = df.fillna(method='ffill')
output = df.to_csv('output.csv',index=False)
os.remove("tmp.csv")

In [14]:
df

,rtss-curr_time,rtss-line-id,rtss-line-stations-id,rtss-line-stations-track-id,rtss-line-stations-track-train-seq,rtss-line-stations-track-train-dest,rtss-line-stations-track-train-plat,rtss-line-stations-track-train-time,rtss-line-stations-track-train-ttnt,rtss-line-stations-track-train-valid,rtss-line-stations-track-train-td,rtss-line-stations-track-train-source
0,2020-12-14 15:00:00,WRL,TUM,DOWN,1,HUH,2,2020-12-14 14:59:40,0,Y,HUH011,-
1,2020-12-14 15:00:00,WRL,TUM,DOWN,2,HUH,1,2020-12-14 15:06:40,7,Y,TUM012,-
2,2020-12-14 15:00:00,WRL,TUM,DOWN,3,HUH,2,2020-12-14 15:13:40,14,Y,TUM001,-
3,2020-12-14 15:00:00,WRL,TUM,DOWN,4,HUH,2,2020-12-14 15:20:40,21,Y,TUM002,-
4,2020-12-14 15:00:00,WRL,TUM,DOWN,5,HUH,2,2020-12-14 15:27:40,28,Y,TUM003,-
...,...,...,...,...,...,...,...,...,...,...,...,...
322433,2020-12-14 18:46:30,WRL,HUH,UP,6,TUM,3,2020-12-14 19:12:10,26,Y,HUH007,-
322434,2020-12-14 18:46:30,WRL,HUH,UP,7,TUM,2,2020-12-14 19:15:40,29,Y,HUH035,-
322435,2020-12-14 18:46:30,WRL,HUH,UP,8,TUM,2,2020-12-14 19:22:40,36,Y,HUH036,-
322436,2020-12-14 18:46:30,WRL,HUH,UP,9,TUM,2,2020-12-14 19:24:04,38,Y,TUM220,-
